- 📺 YouTube video: https://youtu.be/wM-KP_wNAeY

- 📹 Bilibili: https://www.bilibili.com/video/BV1P9tizcEKD/

 # Building Qwen3 from Scratch: A Complete Tutorial



 Welcome to this comprehensive tutorial where we'll build a Qwen3-style language model from scratch!

 We'll implement Grouped-Query Attention (GQA), RMSNorm, SwiGLU activations, and add our own spin - new Muon optimizer that accelerates training by 30% to 50%.



 **What you'll learn:**

🧠 Modern Transformer architecture with Qwen3-style features

⚙️ Grouped-Query Attention (GQA) for memory and compute efficiency

💡 Rotary Positional Embeddings (RoPE) for better performance and context window extrapolation

📐 QK-Norm with RMSNorm for improved numerical / training stability

🚀 Muon optimizer using Newton-Schulz orthogonalization for better weight updates, faster learning with less data

🔁 Hybrid optimization using Muon for matrices and AdamW for other parameters

🔄 SwiGLU activation and deep residual learning in the feedforward layers

🔢 Efficient dataset tokenization and caching with HuggingFace Datasets and Transformers

🧪 Validation metrics including loss, accuracy, and perplexity

🧵 Gradient accumulation + AMP (Automatic Mixed Precision) training for larger batch sizes

🎛️ Cosine learning rate scheduling with warmup

### 📌 Focus: **Qwen-Specific Architecture**

Before diving deep into Qwen, it's **highly recommended** to build a solid foundation in the attention mechanism and tokenization.

Here are two excellent resources to get you started:

---

🎓 **[🦙 LLaMA 4 From Scratch (first 2h 30min)](https://youtu.be/wcDV3l4CD14)**

> In the first **2h 30min**, I give a **clear and intuitive explanation** of both:

* 🧠 Attention Mechanism
* 🧩 Tokens & Tokenizers

Highly recommended if you're just starting out or want a solid refresher.

---

🎥 **[📘 GPT From Scratch by Andrej Karpathy](https://youtu.be/kCc8FmEb1nY)**

> A legendary tutorial from Karpathy that walks through building a GPT model from scratch. Great for understanding the fundamentals!

💡 If this notebook asks you for a HF_TOKEN (Hugging Face token to download tokenizer or data), you can just hit Cancel — it's not actually needed.

🤷‍♂️ Not sure how to disable it!

 ## 1. Setup and Imports



 First, let's import all the necessary libraries. We'll use PyTorch for the deep learning framework,

 transformers for tokenization, and various utilities for data handling and training.

In [ ]:
import torch
import torch.nn as nn  # Neural network modules like Linear, Embedding, etc.
import torch.nn.functional as F  # Functional interface for operations like cross_entropy, silu, etc.
from torch.utils.data import Dataset, DataLoader  # Base class and utilities for loading datasets
from torch.cuda.amp import autocast, GradScaler  # 🔄 Automatic Mixed Precision (AMP) tools for faster/lower-memory training

import math  # Standard math operations (e.g. sqrt, exp, cos)
import random  # Python's random number utilities (used for seeding)
import numpy as np  # Numerical computing library, used for random seeding and general array ops

from datasets import load_dataset  # 🧁 Hugging Face Datasets library for streaming large datasets
from tqdm import tqdm  # ⏳ Progress bar visualization library, great for loops

import time  # ⌛ Timing utilities, measuring time
from transformers import AutoTokenizer  # 🤗 Load pretrained tokenizers from HuggingFace with one line

from dataclasses import dataclass  # 🧱 Define simple classes for configs with less boilerplate
from typing import List, Optional  # ✍️ Type hints for better readability and tooling

import warnings  # ⚠️ Suppress or handle warnings
import os  # 🗂️ File system operations (creating folders, path checking, etc.)
import pickle  # 💾 Python object serialization (used to save/load preprocessed datasets)

warnings.filterwarnings('ignore')  # Silences warnings for cleaner outputs during training


 ## 2. Utility Functions



 Let's start with some utility functions for reproducibility and configuration management.

 The `set_seed` function ensures our experiments are reproducible.

In [ ]:
def set_seed(seed: int = 42):
    """Set all random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🌱 Set all seeds to {seed}")


 ## 3. Model Configuration



 Here we define our model configuration using a dataclass. This makes it easy to experiment

 with different model sizes and hyperparameters. Our model will be a smaller version of Qwen3

 with 384 dimensions, 6 layers, and 8 attention heads.

In [ ]:
@dataclass
class ModelConfig:
    # Model architecture
    d_model: int = 384
    n_heads: int = 8
    n_layers: int = 6
    d_ff: int = 1536
    batch_size: int = 24
    max_steps: int = 2000

    # Qwen3-like parameters
    n_kv_heads: int = 4  # For Grouped-Query Attention
    sliding_window: int = 4096  # Set a large default, effectively disabling it unless specified
    attention_bias: bool = False  # Qwen3 often sets this to False
    rms_norm_eps: float = 1e-6  # Epsilon for RMSNorm

    # Training parameters
    gradient_accumulation_steps: int = 4
    muon_lr: float = 0.01

    # Data parameters
    max_seq_len: int = 512
    num_documents: int = 2000
    max_tokens: int = 500000

    # Evaluation
    eval_every: int = 500
    eval_steps: int = 100

    # Regularization
    weight_decay: float = 0.1
    dropout: float = 0.1
    grad_clip: float = 1.0

    # Technical
    use_amp: bool = True
    vocab_size: Optional[int] = None

    def __post_init__(self):
        self.d_k = self.d_model // self.n_heads
        assert self.d_model % self.n_heads == 0, "d_model must be divisible by n_heads"
        assert self.n_heads % self.n_kv_heads == 0, "n_heads must be divisible by n_kv_heads"
        self.n_kv_groups = self.n_heads // self.n_kv_heads


 ## 4. Grouped-Query Attention Helper



 This function implements the key component of GQA - repeating key and value heads.

 In GQA, we have fewer key-value heads than query heads, which reduces memory usage

 while maintaining performance.

In [ ]:
def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep).
    The hidden states go from (batch, num_key_value_heads, seqlen, head_dim)
    to (batch, num_attention_heads, seqlen, head_dim)
    """
    # Extract dimensions from input tensor
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape

    # Early return if no repetition is needed
    if n_rep == 1:
        return hidden_states

    # Add a new dimension at index 2 (after num_key_value_heads) and expand
    # Shape transformation:
    # (batch, num_key_value_heads, slen, head_dim)
    # -> (batch, num_key_value_heads, 1, slen, head_dim) [via None indexing]
    # -> (batch, num_key_value_heads, n_rep, slen, head_dim) [via expand]
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)

    # Flatten the num_key_value_heads and n_rep dimensions together
    # Final shape: (batch, num_key_value_heads * n_rep, slen, head_dim)
    # This effectively repeats each key/value head n_rep times
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)

#### Optional: Code excercises to understand `repeat_kv`

In [ ]:
# Tensor Expansion and Repetition Practice Exercises

import torch
import numpy as np

print("🚀 Tensor Expansion and Repetition Exercises")
print("=" * 50)

# =============================================================================
# EXERCISE 1: Basic Tensor Creation and Shapes
# =============================================================================
print("\n📝 Exercise 1: Understanding Tensor Shapes")
print("-" * 40)

# Create simple tensors and understand their shapes
x = torch.tensor([1, 2, 3])
print(f"1D tensor: {x}")
print(f"Shape: {x.shape}")

y = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(f"2D tensor:\n{y}")
print(f"Shape: {y.shape}")

z = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
print(f"3D tensor:\n{z}")
print(f"Shape: {z.shape}")

# TODO: Create a 4D tensor of shape (2, 3, 4, 5) filled with ones
# Your code here:
# tensor_4d = ?

🚀 Tensor Expansion and Repetition Exercises

📝 Exercise 1: Understanding Tensor Shapes
----------------------------------------
1D tensor: tensor([1, 2, 3])
Shape: torch.Size([3])
2D tensor:
tensor([[1, 2, 3],
        [4, 5, 6]])
Shape: torch.Size([2, 3])
3D tensor:
tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
Shape: torch.Size([2, 2, 2])


In [ ]:
# =============================================================================
# EXERCISE 2: Understanding None Indexing (Adding Dimensions)
# =============================================================================
print("\n\n📝 Exercise 2: Adding Dimensions with None")
print("-" * 40)

# Start with a 1D tensor
a = torch.tensor([1, 2, 3, 4])
print(f"Original: {a.shape} -> {a}")

# Add dimension at different positions
a_new_dim0 = a[None, :]  # or a.unsqueeze(0)
print(f"Add dim at 0: {a_new_dim0.shape} -> {a_new_dim0}")

a_new_dim1 = a[:, None]  # or a.unsqueeze(1)
print(f"Add dim at 1: {a_new_dim1.shape} -> {a_new_dim1}")

a_new_dim_end = a[..., None]  # or a.unsqueeze(-1)
print(f"Add dim at end: {a_new_dim_end.shape} -> {a_new_dim_end}")

# Multiple dimensions
a_multi = a[None, :, None, None]
print(f"Multiple dims: {a_multi.shape}")

# TODO: Take tensor [1, 2, 3] and make it shape (1, 3, 1, 1)
# Your code here:
# result = ?



📝 Exercise 2: Adding Dimensions with None
----------------------------------------
Original: torch.Size([4]) -> tensor([1, 2, 3, 4])
Add dim at 0: torch.Size([1, 4]) -> tensor([[1, 2, 3, 4]])
Add dim at 1: torch.Size([4, 1]) -> tensor([[1],
        [2],
        [3],
        [4]])
Add dim at end: torch.Size([4, 1]) -> tensor([[1],
        [2],
        [3],
        [4]])
Multiple dims: torch.Size([1, 4, 1, 1])


In [ ]:
# =============================================================================
# EXERCISE 3: Basic expand() Operation
# =============================================================================
print("\n\n📝 Exercise 3: Understanding expand()")
print("-" * 40)

# expand() creates a view with repeated elements (no memory copy!)
b = torch.tensor([[1, 2, 3]])  # Shape: (1, 3)
print(f"Original: {b.shape} -> {b}")

# Expand the first dimension
b_expanded = b.expand(4, 3)  # Repeat the row 4 times
print(f"Expanded: {b_expanded.shape}")
print(b_expanded)

# Expand with -1 (keep original size)
c = torch.tensor([[1], [2], [3]])  # Shape: (3, 1)
print(f"\nOriginal c: {c.shape}")
print(c)

c_expanded = c.expand(-1, 5)  # Keep dim 0, expand dim 1 to 5
print(f"Expanded c: {c_expanded.shape}")
print(c_expanded)

# TODO: Create tensor [[1, 2]] and expand it to shape (3, 4)
# Your code here:
# d = ?
# d_expanded = ?



📝 Exercise 3: Understanding expand()
----------------------------------------
Original: torch.Size([1, 3]) -> tensor([[1, 2, 3]])
Expanded: torch.Size([4, 3])
tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

Original c: torch.Size([3, 1])
tensor([[1],
        [2],
        [3]])
Expanded c: torch.Size([3, 5])
tensor([[1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]])


In [ ]:
# =============================================================================
# EXERCISE 4: repeat() vs expand() vs repeat_interleave()
# =============================================================================
print("\n\n📝 Exercise 4: Different Repetition Methods")
print("-" * 40)

original = torch.tensor([1, 2, 3])
print(f"Original: {original}")

# Method 1: repeat() - actually copies data
repeated = original.repeat(2)  # Repeat entire tensor 2 times
print(f"repeat(2): {repeated}")

repeated_2d = original.repeat(2, 1)  # 2D repetition
print(f"repeat(2, 1) on [1,2,3]: shape {repeated_2d.shape}")
print(repeated_2d)

# Method 2: expand() - creates view (memory efficient)
original_2d = original.unsqueeze(0)  # Make it (1, 3)
expanded = original_2d.expand(3, -1)
print(f"expand(3, -1): shape {expanded.shape}")
print(expanded)

# Method 3: repeat_interleave() - repeats each element
interleaved = torch.repeat_interleave(original, 2)
print(f"repeat_interleave(2): {interleaved}")

# TODO: What's the difference between these results?
# torch.tensor([1, 2]).repeat(3) vs torch.repeat_interleave(torch.tensor([1, 2]), 3)



📝 Exercise 4: Different Repetition Methods
----------------------------------------
Original: tensor([1, 2, 3])
repeat(2): tensor([1, 2, 3, 1, 2, 3])
repeat(2, 1) on [1,2,3]: shape torch.Size([2, 3])
tensor([[1, 2, 3],
        [1, 2, 3]])
expand(3, -1): shape torch.Size([3, 3])
tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])
repeat_interleave(2): tensor([1, 1, 2, 2, 3, 3])


In [ ]:
# =============================================================================
# EXERCISE 5: Working with 3D Tensors
# =============================================================================
print("\n\n📝 Exercise 5: 3D Tensor Manipulations")
print("-" * 40)

# Create a 3D tensor: (batch=2, heads=3, features=4)
tensor_3d = torch.arange(24).reshape(2, 3, 4)
print(f"3D tensor shape: {tensor_3d.shape}")
print(f"3D tensor:\n{tensor_3d}")

# Add a dimension in the middle
tensor_4d = tensor_3d[:, :, None, :]  # Shape: (2, 3, 1, 4)
print(f"\nAfter adding dim: {tensor_4d.shape}")

# Expand the new dimension
tensor_expanded = tensor_4d.expand(2, 3, 5, 4)  # Shape: (2, 3, 5, 4)
print(f"After expand: {tensor_expanded.shape}")
print(f"First batch, first head:\n{tensor_expanded[0, 0]}")

# TODO: Take the expanded tensor and reshape it to merge dimensions 1 and 2
# Target shape: (2, 15, 4)  # 3 * 5 = 15
# Your code here:
# merged = ?



📝 Exercise 5: 3D Tensor Manipulations
----------------------------------------
3D tensor shape: torch.Size([2, 3, 4])
3D tensor:
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

After adding dim: torch.Size([2, 3, 1, 4])
After expand: torch.Size([2, 3, 5, 4])
First batch, first head:
tensor([[0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3],
        [0, 1, 2, 3]])


In [ ]:
# =============================================================================
# EXERCISE 6: Simulating the repeat_kv Pattern
# =============================================================================
print("\n\n📝 Exercise 6: Building Up to repeat_kv")
print("-" * 40)

# Simulate key/value heads that need to be repeated
# Shape: (batch, num_kv_heads, seq_len, head_dim)
kv_tensor = torch.arange(48).reshape(2, 3, 4, 2)
print(f"KV tensor shape: {kv_tensor.shape}")
print(f"KV tensor (batch 0, head 0):\n{kv_tensor[0, 0]}")

n_rep = 2  # Each KV head needs to be repeated 2 times

# Step 1: Add dimension for repetition
step1 = kv_tensor[:, :, None, :, :]  # Shape: (2, 3, 1, 4, 2)
print(f"\nStep 1 - Add dimension: {step1.shape}")

# Step 2: Expand the new dimension
step2 = step1.expand(2, 3, n_rep, 4, 2)  # Shape: (2, 3, 2, 4, 2)
print(f"Step 2 - Expand: {step2.shape}")

# Step 3: Reshape to merge heads
final = step2.reshape(2, 3 * n_rep, 4, 2)  # Shape: (2, 6, 4, 2)
print(f"Step 3 - Final: {final.shape}")

# Verify: each original head should appear n_rep times
print(f"\nOriginal head 0:\n{kv_tensor[0, 0]}")
print(f"Repeated head 0 (position 0):\n{final[0, 0]}")
print(f"Repeated head 0 (position 1):\n{final[0, 1]}")
print(f"Original head 1:\n{kv_tensor[0, 1]}")
print(f"Repeated head 1 (position 2):\n{final[0, 2]}")

# TODO: Verify that final[0, 0] equals final[0, 1] (same repeated head)



📝 Exercise 6: Building Up to repeat_kv
----------------------------------------
KV tensor shape: torch.Size([2, 3, 4, 2])
KV tensor (batch 0, head 0):
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7]])

Step 1 - Add dimension: torch.Size([2, 3, 1, 4, 2])
Step 2 - Expand: torch.Size([2, 3, 2, 4, 2])
Step 3 - Final: torch.Size([2, 6, 4, 2])

Original head 0:
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7]])
Repeated head 0 (position 0):
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7]])
Repeated head 0 (position 1):
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7]])
Original head 1:
tensor([[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]])
Repeated head 1 (position 2):
tensor([[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]])


In [ ]:
# =============================================================================
# EXERCISE 7: Complete repeat_kv Implementation Practice
# =============================================================================
print("\n\n📝 Exercise 7: Implement repeat_kv Yourself")
print("-" * 40)

def my_repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    TODO: Implement this function!
    Input: (batch, num_key_value_heads, seqlen, head_dim)
    Output: (batch, num_key_value_heads * n_rep, seqlen, head_dim)
    """
    # Get dimensions
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape

    # Handle n_rep = 1 case
    if n_rep == 1:
        return hidden_states

    # TODO: Your implementation here
    # Step 1: Add dimension
    # Step 2: Expand
    # Step 3: Reshape
    # return ?
    pass

# Test your implementation
test_tensor = torch.arange(24).reshape(1, 3, 4, 2)
print(f"Test input shape: {test_tensor.shape}")

# TODO: Uncomment when you implement the function
# result = my_repeat_kv(test_tensor, 2)
# print(f"Result shape: {result.shape}")



📝 Exercise 7: Implement repeat_kv Yourself
----------------------------------------
Test input shape: torch.Size([1, 3, 4, 2])


In [ ]:
# =============================================================================
# EXERCISE 8: Advanced Patterns
# =============================================================================
print("\n\n📝 Exercise 8: Advanced Repetition Patterns")
print("-" * 40)

# Pattern 1: Repeat different amounts for different dimensions
base = torch.tensor([[1, 2], [3, 4]])  # Shape: (2, 2)
print(f"Base tensor:\n{base}")

# Repeat 3 times along dim 0, 2 times along dim 1
pattern1 = base.repeat(3, 2)
print(f"Pattern 1 - repeat(3, 2):\n{pattern1}")

# Pattern 2: Using repeat_interleave along specific dimensions
pattern2 = torch.repeat_interleave(base, 2, dim=0)
print(f"Pattern 2 - repeat_interleave along dim 0:\n{pattern2}")

pattern3 = torch.repeat_interleave(base, 2, dim=1)
print(f"Pattern 3 - repeat_interleave along dim 1:\n{pattern3}")

# TODO: Create a pattern where you repeat_interleave along dim 0 with repeats [1, 3]
# (first row once, second row three times)



📝 Exercise 8: Advanced Repetition Patterns
----------------------------------------
Base tensor:
tensor([[1, 2],
        [3, 4]])
Pattern 1 - repeat(3, 2):
tensor([[1, 2, 1, 2],
        [3, 4, 3, 4],
        [1, 2, 1, 2],
        [3, 4, 3, 4],
        [1, 2, 1, 2],
        [3, 4, 3, 4]])
Pattern 2 - repeat_interleave along dim 0:
tensor([[1, 2],
        [1, 2],
        [3, 4],
        [3, 4]])
Pattern 3 - repeat_interleave along dim 1:
tensor([[1, 1, 2, 2],
        [3, 3, 4, 4]])


In [ ]:
# =============================================================================
# EXERCISE 9: Memory Efficiency Test
# =============================================================================
print("\n\n📝 Exercise 9: Memory Usage Comparison")
print("-" * 40)

large_tensor = torch.randn(100, 50)
print(f"Original tensor memory: {large_tensor.numel() * large_tensor.element_size()} bytes")

# Method 1: expand (memory efficient - creates view)
expanded = large_tensor.unsqueeze(0).expand(10, -1, -1)
print(f"Expanded tensor shares memory: {expanded.storage().data_ptr() == large_tensor.storage().data_ptr()}")

# Method 2: repeat (creates copy)
repeated = large_tensor.repeat(10, 1)
print(f"Repeated tensor shares memory: {repeated.storage().data_ptr() == large_tensor.storage().data_ptr()}")

# TODO: What happens if you modify the original tensor? Will the expanded version change too?



📝 Exercise 9: Memory Usage Comparison
----------------------------------------
Original tensor memory: 20000 bytes
Expanded tensor shares memory: True
Repeated tensor shares memory: False


In [ ]:
# =============================================================================
# EXERCISE 10: Real-World Application - Attention Mechanism
# =============================================================================
print("\n\n📝 Exercise 10: Attention Mechanism Context")
print("-" * 40)

# Simulate a real attention scenario
batch_size = 2
seq_len = 8
num_query_heads = 12
num_kv_heads = 4  # Fewer KV heads than query heads (Grouped Query Attention)
head_dim = 64

print(f"Scenario: {num_query_heads} query heads, {num_kv_heads} KV heads")
print(f"Need to repeat each KV head {num_query_heads // num_kv_heads} times")

# Create mock key and value tensors
keys = torch.randn(batch_size, num_kv_heads, seq_len, head_dim)
values = torch.randn(batch_size, num_kv_heads, seq_len, head_dim)

print(f"Original keys shape: {keys.shape}")
print(f"Original values shape: {values.shape}")

# Apply repeat_kv to match query heads
n_rep = num_query_heads // num_kv_heads

def repeat_kv_solution(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)

keys_repeated = repeat_kv_solution(keys, n_rep)
values_repeated = repeat_kv_solution(values, n_rep)

print(f"Repeated keys shape: {keys_repeated.shape}")
print(f"Repeated values shape: {values_repeated.shape}")

# TODO: Verify that we now have the same number of heads for keys, values, and queries
print(f"Success! KV heads now match query heads: {keys_repeated.shape[1] == num_query_heads}")

print("\n🎉 Exercises Complete!")
print("Next steps:")
print("1. Try modifying the dimensions and see how shapes change")
print("2. Experiment with different n_rep values")
print("3. Compare memory usage between expand() and repeat()")
print("4. Implement your own version of repeat_kv from scratch!")



📝 Exercise 10: Attention Mechanism Context
----------------------------------------
Scenario: 12 query heads, 4 KV heads
Need to repeat each KV head 3 times
Original keys shape: torch.Size([2, 4, 8, 64])
Original values shape: torch.Size([2, 4, 8, 64])
Repeated keys shape: torch.Size([2, 12, 8, 64])
Repeated values shape: torch.Size([2, 12, 8, 64])
Success! KV heads now match query heads: True

🎉 Exercises Complete!
Next steps:
1. Try modifying the dimensions and see how shapes change
2. Experiment with different n_rep values
3. Compare memory usage between expand() and repeat()
4. Implement your own version of repeat_kv from scratch!


 ## 5. Muon Optimizer - The Secret Sauce



 The Muon optimizer is a novel approach that uses Newton-Schulz iteration for orthogonalization.

 This helps with training stability and convergence. The `zeropower_via_newtonschulz5` function

 implements the core mathematical operation.

To really *understand* the **Muon optimizer** (🔥 the best new optimizers), I recommend checking out these tutorials:

- 🔁 [Backpropagation From Scratch](https://youtu.be/W8g1hvW4Wic) — Understand gradients deeply
- 🧠 [Orthonormal Matrix Intuition](https://youtu.be/FbYRZpBgFz4) — Key concept behind Muon’s update step

These will level up your optimizer intuition 💪✨

In [ ]:
@torch.compile
def zeropower_via_newtonschulz5(G: torch.Tensor, steps: int = 5) -> torch.Tensor:
    """Newton-Schulz iteration to compute the zeroth power / orthogonalization of G."""
    assert G.ndim >= 2
    a, b, c = (3.4445, -4.7750, 2.0315)
    X = G.bfloat16()

    if G.size(-2) > G.size(-1):
        X = X.mT

    X = X / (X.norm(dim=(-2, -1), keepdim=True) + 1e-7)

    for _ in range(steps):
        A = X @ X.mT
        B = b * A + c * A @ A
        X = a * X + B @ X

    if G.size(-2) > G.size(-1):
        X = X.mT

    return X

class Muon(torch.optim.Optimizer):
    """Muon - MomentUm Orthogonalized by Newton-schulz"""
    def __init__(self, params, lr=0.02, momentum=0.95, nesterov=True, ns_steps=5):
        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov, ns_steps=ns_steps)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue

                g = p.grad
                state = self.state[p]

                # Initialize momentum buffer if first time
                if "momentum_buffer" not in state:
                    state["momentum_buffer"] = torch.zeros_like(g)

                buf = state["momentum_buffer"]
                # Update momentum buffer: buf = momentum * buf + (1-momentum) * grad
                buf.lerp_(g, 1 - group["momentum"])
                # Apply Nesterov momentum if enabled, otherwise use standard momentum
                g = g.lerp_(buf, group["momentum"]) if group["nesterov"] else buf
                # Apply zero-power normalization via Newton-Schulz iterations (make it close to orthonormal)
                g = zeropower_via_newtonschulz5(g, steps=group["ns_steps"])
                # Update parameters with adaptive scaling based on parameter shape
                p.add_(g.view_as(p), alpha=-group["lr"] * max(1, p.size(-2) / p.size(-1))**0.5)
                # Updates parameters with an adaptive learning rate that scales based on the parameter tensor's aspect ratio (height/width). For matrices where height > width, it increases the effective learning rate by √(height/width)

 ## 6. Data Loading and Caching



 Loading and processing data can be time-consuming. We implement caching to avoid

 reprocessing the same data multiple times. This function loads the SmolLM corpus

 and tokenizes it efficiently.

In [ ]:
def load_and_cache_data(config: ModelConfig, cache_dir: str = "data_cache"):
    """Load and cache tokenized data to avoid reprocessing"""
    os.makedirs(cache_dir, exist_ok=True)
    cache_file = f"{cache_dir}/tokenized_data_{config.num_documents}_{config.max_tokens}.pkl"

    # Check if cached data exists
    if os.path.exists(cache_file):
        print(f"📦 Loading cached data from {cache_file}")
        with open(cache_file, 'rb') as f:
            cached_data = pickle.load(f)

        texts = cached_data['texts']
        tokenizer = cached_data['tokenizer']
        tokens = cached_data['tokens']
        config.vocab_size = tokenizer.vocab_size

        print(f"✅ Loaded {len(texts)} documents, {len(tokens):,} tokens from cache")
        return texts, tokenizer, tokens

    print(f"🔄 Processing new data (will cache for future use)")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load dataset
    dataset = load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", split="train", streaming=True)

    texts = []
    for i, item in enumerate(dataset):
        if i >= config.num_documents:
            break
        texts.append(item["text"][:3000])

    print(f"Loaded {len(texts)} documents")

    # Tokenize
    print("Tokenizing texts...")
    all_tokens = []
    for text in tqdm(texts, desc="Tokenizing"):
        tokens = tokenizer.encode(text, add_special_tokens=False)
        all_tokens.extend(tokens)

    tokens = all_tokens[:config.max_tokens]
    print(f"Using {len(tokens):,} tokens")
    config.vocab_size = tokenizer.vocab_size

    # Cache the processed data
    cached_data = {'texts': texts, 'tokenizer': tokenizer, 'tokens': tokens}
    with open(cache_file, 'wb') as f:
        pickle.dump(cached_data, f)

    print(f"💾 Cached data to {cache_file}")
    return texts, tokenizer, tokens


 ## 7. Dataset Class



 We create a custom dataset class for language modeling. This creates sliding windows

 of tokens for training, where each sample is a sequence and its corresponding target

 (shifted by one position).

In [ ]:
class TextTokenDataset(Dataset):
    def __init__(self, tokens: List[int], seq_len: int = 512):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return max(0, len(self.tokens) - self.seq_len)

    def __getitem__(self, idx):
        x = torch.tensor(self.tokens[idx:idx + self.seq_len], dtype=torch.long)
        y = torch.tensor(self.tokens[idx + 1:idx + self.seq_len + 1], dtype=torch.long)
        return x, y


💬 [ChatGPT explanation and excercise](https://chatgpt.com/share/689455f2-4598-8002-9ac2-f8c3913ecad7)

 ## 8. Rotary Position Embeddings (RoPE)



 RoPE is a modern alternative to positional encodings that allows the model to

 generalize to longer sequences. It applies rotation matrices to the embeddings

 based on their position.

My videos explaining RoPE (first is most important)

- 📌 **[Rotary Positional Embeddings & Rotation Matrix + Python LLM Code](https://youtu.be/wiJ-OU-URYg)**
- 🧠 **[Get SMARTER Than 99% of AI Researchers](https://youtu.be/X0JryI85hL0)** - Beginning part
- 🛠️ **[RoPE In DeepSeek V3 – Code Step by Step](https://youtu.be/Rs9tLDSMUkM)**


In [ ]:
class Rotary(nn.Module):
    def __init__(self, dim: int, max_seq_len: int):
        super().__init__()
        angular_freq = (1 / 10000) ** torch.linspace(0, 1, steps=dim//4, dtype=torch.float32)
        angular_freq = torch.cat([angular_freq, angular_freq.new_zeros(dim//4)])
        t = torch.arange(max_seq_len, dtype=torch.float32)
        theta = torch.einsum("i,j -> ij", t, angular_freq)
        self.register_buffer('cos', theta.cos(), persistent=False)
        self.register_buffer('sin', theta.sin(), persistent=False)

    def forward(self, x_BTHD: torch.Tensor):
        assert self.cos.size(0) >= x_BTHD.size(-3)
        cos, sin = self.cos[None, :x_BTHD.size(-3), None, :], self.sin[None, :x_BTHD.size(-3), None, :]
        x1, x2 = x_BTHD.to(dtype=torch.float32).chunk(2, dim=-1)
        y1 = x1 * cos + x2 * sin
        y2 = x1 * (-sin) + x2 * cos
        return torch.cat((y1, y2), 3).type_as(x_BTHD)


[Excercises with ChatGPT](https://chatgpt.com/share/68945a01-8d48-8002-8cf0-04b7f6db744b)

 ## 9. Grouped-Query Attention Implementation



 This is the heart of our model - the attention mechanism with GQA. Notice how we:

 1. Project Q, K, V separately

 2. Apply QK normalization (a Qwen3 innovation)

 3. Use RoPE for positional information

 4. Implement GQA by repeating K and V heads

 5. Use scaled dot-product attention

In [ ]:
class Qwen3Attention(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.d_model = config.d_model
        self.n_heads = config.n_heads
        self.n_kv_heads = config.n_kv_heads
        self.n_kv_groups = config.n_kv_groups
        self.d_k = config.d_k

        # Separate linear layers for Q, K, V
        self.q_proj = nn.Linear(self.d_model, self.n_heads * self.d_k, bias=config.attention_bias)
        self.k_proj = nn.Linear(self.d_model, self.n_kv_heads * self.d_k, bias=config.attention_bias)
        self.v_proj = nn.Linear(self.d_model, self.n_kv_heads * self.d_k, bias=config.attention_bias)
        self.w_o = nn.Linear(self.d_model, self.d_model, bias=False)

        # QK-Normalization layers
        # Practice RMSNorm 1 on 1 with ChatGPT - https://chatgpt.com/share/68945c86-2dd4-8002-b017-725caab0c107
        self.q_norm = nn.RMSNorm(self.d_k, eps=config.rms_norm_eps)
        self.k_norm = nn.RMSNorm(self.d_k, eps=config.rms_norm_eps)

        self.rotary = Rotary(self.d_k, config.max_seq_len)
        self.dropout = config.dropout

    def forward(self, x):
        batch_size, seq_len = x.size(0), x.size(1)

        # 1. Project Q, K, V separately
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)

        # 2. Reshape into heads
        q = q.view(batch_size, seq_len, self.n_heads, self.d_k)
        k = k.view(batch_size, seq_len, self.n_kv_heads, self.d_k)
        v = v.view(batch_size, seq_len, self.n_kv_heads, self.d_k)

        # 3. Apply QK-Norm
        q = self.q_norm(q)
        k = self.k_norm(k)

        # 4. Apply RoPE
        # Transpose to (batch, seq_len, n_heads, d_k) -> (batch, n_heads, seq_len, d_k) for rotary
        q = self.rotary(q.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)
        k = self.rotary(k.permute(0, 2, 1, 3)).permute(0, 2, 1, 3)

        # Transpose for attention: (batch, seq_len, n_heads, d_k) -> (batch, n_heads, seq_len, d_k)
        Q = q.transpose(1, 2)
        K = k.transpose(1, 2)
        V = v.transpose(1, 2)

        # 5. Repeat K and V heads for GQA
        K = repeat_kv(K, self.n_kv_groups)
        V = repeat_kv(V, self.n_kv_groups)

        # 6. Scaled Dot-Product Attention
        attn_output = F.scaled_dot_product_attention(
            Q, K, V, is_causal=True, dropout_p=self.dropout if self.training else 0.0
        )

        # 7. Reshape and final projection
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        return self.w_o(attn_output)


 ## 10. SwiGLU Feed-Forward Network



 SwiGLU is a modern activation function that combines Swish and GLU. It's more

 effective than traditional ReLU and is used in many modern models including Qwen3.

In [ ]:
class SwiGLUFeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.gate_proj = nn.Linear(d_model, d_ff, bias=False)
        self.down_proj = nn.Linear(d_ff, d_model, bias=False)
        self.up_proj = nn.Linear(d_model, d_ff, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Implementation of the SwiGLU activation function
        # F.silu is the Swish activation function
        activated_x = F.silu(self.gate_proj(x)) * self.up_proj(x)
        return self.down_proj(self.dropout(activated_x))


Think of:

`output = gate(x) * value(x)`

like:

`light = brightness_control × light_source`


 ## 11. Transformer Block



 Each transformer block combines attention and feed-forward layers with residual

 connections and normalization. We use RMSNorm instead of LayerNorm for better

 training stability.

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, config: ModelConfig):  # Pass the entire config object
        super().__init__()
        self.attention = Qwen3Attention(config)
        self.feed_forward = SwiGLUFeedForward(config.d_model, config.d_ff, config.dropout)
        self.norm1 = nn.RMSNorm(config.d_model, eps=config.rms_norm_eps)
        self.norm2 = nn.RMSNorm(config.d_model, eps=config.rms_norm_eps)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        attn_out = self.attention(self.norm1(x))
        x = x + self.dropout(attn_out)
        ff_out = self.feed_forward(self.norm2(x))
        x = x + self.dropout(ff_out)
        return x


 ## 12. Complete Language Model



 Now we assemble everything into our complete language model. This includes:

 - Token embeddings

 - Positional dropout

 - Stack of transformer blocks

 - Final normalization and output projection

 - Weight tying between input embeddings and output layer

In [ ]:
class MinimalLLM(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config

        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_dropout = nn.Dropout(config.dropout)

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(config) for _ in range(config.n_layers)
        ])

        self.norm = nn.RMSNorm(config.d_model, eps=config.rms_norm_eps)
        self.output_dropout = nn.Dropout(config.dropout)

        # Tie weights
        # This ties the output layer (`lm_head`) weights to the input token embedding weights so the model shares parameters between input and output, reducing memory and improving generalization.
        # https://chatgpt.com/share/6894683e-ba44-8002-ae82-e42b4afc9d98
        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)
        self.lm_head.weight = self.token_embedding.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x):
        x = self.token_embedding(x) * math.sqrt(self.config.d_model)
        x = self.position_dropout(x)

        for block in self.transformer_blocks:
            x = block(x)

        x = self.norm(x)
        x = self.output_dropout(x)
        logits = self.lm_head(x)
        return logits


 ## 13. Evaluation Function



 During training, we need to evaluate our model's performance. This function

 computes loss, accuracy, and perplexity on the validation set.

In [ ]:
def evaluate_model(model: nn.Module, val_loader: DataLoader, config: ModelConfig):
    """Evaluate model performance"""
    model.eval()
    total_loss = 0
    total_tokens = 0
    total_correct = 0

    device = next(model.parameters()).device

    with torch.no_grad():  # Disable gradient computation for evaluation (saves memory and computation)
        for i, (x, y) in enumerate(val_loader):
            # Stop evaluation after specified number of steps to limit eval time
            if i >= config.eval_steps:
                break

            # Move input sequences (x) and target sequences (y) to GPU/device
            x, y = x.to(device), y.to(device)

            # Use automatic mixed precision if enabled (faster training with minimal accuracy loss)
            with autocast(enabled=config.use_amp):
                # Forward pass: get model predictions (logits) for input sequence
                logits = model(x)

                # Calculate cross-entropy loss between predictions and targets
                # Reshape to (batch_size * seq_len, vocab_size) and (batch_size * seq_len,)
                # for proper cross-entropy computation across all token positions
                loss = F.cross_entropy(logits.view(-1, config.vocab_size), y.view(-1))

            # Accumulate total loss weighted by number of tokens in this batch
            total_loss += loss.item() * y.numel()
            # Keep track of total number of tokens processed
            total_tokens += y.numel()

            # Get predicted token IDs by taking argmax over vocabulary dimension
            predictions = logits.argmax(dim=-1)
            # Count correct predictions for accuracy calculation
            total_correct += (predictions == y).sum().item()

    avg_loss = total_loss / total_tokens
    accuracy = total_correct / total_tokens
    perplexity = math.exp(min(avg_loss, 20))

    model.train()
    return {'val_loss': avg_loss, 'val_accuracy': accuracy, 'val_perplexity': perplexity}


 ## 14. Optimizer Setup



 We use a hybrid approach: Muon optimizer for 2D parameters (attention and feed-forward weights)

 and AdamW for other parameters. This gives us the benefits of both optimizers.

In [ ]:
def setup_muon_optimizer(model: nn.Module, config: ModelConfig):
    """Setup Muon optimizer with hybrid approach"""
    muon_params = []
    adamw_params = []

    for name, param in model.named_parameters():
        if (param.ndim == 2 and
            'token_embedding' not in name and
            'norm' not in name and
            param.requires_grad):
            muon_params.append(param)
        else:
            adamw_params.append(param)

    print(f"  Muon parameters: {sum(p.numel() for p in muon_params):,}")
    print(f"  AdamW parameters: {sum(p.numel() for p in adamw_params):,}")

    muon_optimizer = Muon(muon_params, lr=config.muon_lr, momentum=0.95)
    adamw_optimizer = torch.optim.AdamW(adamw_params, lr=config.muon_lr*0.1, weight_decay=config.weight_decay)

    return [muon_optimizer, adamw_optimizer]


 ## 15. Training Loop



 This is where the magic happens! Our training loop includes:

 - Gradient accumulation for larger effective batch sizes

 - Mixed precision training for speed

 - Learning rate scheduling with warmup and cosine decay

 - Regular evaluation and model checkpointing

 - Progress tracking with detailed metrics

In [ ]:
def train_model(config: ModelConfig, train_loader: DataLoader, val_loader: DataLoader):
    """Train the model with Muon optimizer"""
    print(f"\n🚀 Training Small model with Muon optimizer")

    # Initialize model
    set_seed(42)
    model = MinimalLLM(config)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    total_params = sum(p.numel() for p in model.parameters())
    print(f"  📊 Total parameters: {total_params:,}")

    # Setup optimizers
    optimizers = setup_muon_optimizer(model, config)

    # Learning rate schedule
    schedulers = []
    for optimizer in optimizers:
        warmup_steps = config.max_steps // 20
        def lr_lambda(step):
            if step < warmup_steps:
                return step / warmup_steps
            else:
                progress = (step - warmup_steps) / (config.max_steps - warmup_steps)
                return 0.1 + 0.9 * 0.5 * (1 + math.cos(math.pi * progress))

        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
        schedulers.append(scheduler)

    scaler = GradScaler() if config.use_amp else None

    # Training loop
    model.train()
    step = 0
    start_time = time.time()
    best_val_loss = float('inf')

    pbar = tqdm(total=config.max_steps, desc="Training")

    while step < config.max_steps:
        for batch_idx, (x, y) in enumerate(train_loader):
            if step >= config.max_steps:
                break

            x, y = x.to(device), y.to(device)

            # Forward pass with gradient accumulation
            if config.use_amp:
                with autocast():
                    logits = model(x)
                    loss = F.cross_entropy(logits.view(-1, config.vocab_size), y.view(-1))
                    loss = loss / config.gradient_accumulation_steps
                scaler.scale(loss).backward()
            else:
                logits = model(x)
                loss = F.cross_entropy(logits.view(-1, config.vocab_size), y.view(-1))
                loss = loss / config.gradient_accumulation_steps
                loss.backward()

            # Optimizer step after accumulation
            if (step + 1) % config.gradient_accumulation_steps == 0:
                if config.use_amp:
                    for optimizer in optimizers:
                        scaler.unscale_(optimizer)
                    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)

                    for optimizer in optimizers:
                        scaler.step(optimizer)
                        optimizer.zero_grad()
                    for scheduler in schedulers:
                        scheduler.step()
                    scaler.update()
                else:
                    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
                    for optimizer in optimizers:
                        optimizer.step()
                        optimizer.zero_grad()
                    for scheduler in schedulers:
                        scheduler.step()

            # Logging
            if step % 10 == 0:
                with torch.no_grad():
                    predictions = logits.argmax(dim=-1)
                    accuracy = (predictions == y).float().mean().item()
                    current_loss = loss.item() * config.gradient_accumulation_steps
                    perplexity = math.exp(min(current_loss, 20))

                pbar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'acc': f'{accuracy:.3f}',
                    'ppl': f'{perplexity:.1f}',
                    'lr': f'{optimizers[0].param_groups[0]["lr"]:.2e}'
                })

            # Evaluation
            if step % config.eval_every == 0 and step > 0:
                eval_metrics = evaluate_model(model, val_loader, config)
                print(f"\nStep {step}: Val Loss: {eval_metrics['val_loss']:.4f}, "
                      f"Val Acc: {eval_metrics['val_accuracy']:.4f}, "
                      f"Val PPL: {eval_metrics['val_perplexity']:.2f}")

                if eval_metrics['val_loss'] < best_val_loss:
                    best_val_loss = eval_metrics['val_loss']
                    # Save best model
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'config': config,
                        'step': step,
                        'best_val_loss': best_val_loss,
                        'final_metrics': eval_metrics
                    }, 'best_model.pt')
                    print(f"💾 Saved best model with val_loss: {best_val_loss:.4f}")

            step += 1
            if step % 10 == 0:
                pbar.update(10)

    pbar.close()

    training_time = time.time() - start_time
    print(f"  ⏱️ Training completed in {training_time:.1f} seconds")

    # Final evaluation
    final_eval = evaluate_model(model, val_loader, config)
    print(f"  📊 Final - Loss: {final_eval['val_loss']:.4f}, "
          f"Acc: {final_eval['val_accuracy']:.4f}, PPL: {final_eval['val_perplexity']:.2f}")

    # Save final model
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': config,
        'step': step,
        'final_metrics': final_eval
    }, 'final_model.pt')
    print(f"💾 Saved final model to final_model.pt")

    return model, final_eval


 ## 16. Main Training Script



 Finally, let's put everything together! This section:

 1. Checks system resources

 2. Sets up configuration

 3. Loads and prepares data

 4. Trains the model

 5. Reports final results

In [ ]:
if __name__ == "__main__":
    # Check system
    print(f"🔍 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name()}")
        print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # Set seed
    set_seed(42)

    # Create config for Small model
    config = ModelConfig()
    print(f"\n📋 Model Configuration:")
    print(f"   Architecture: {config.d_model}d, {config.n_layers}L, {config.n_heads}H, {config.d_ff}ff")
    print(f"   Training: {config.max_steps} steps, batch size {config.batch_size}")
    print(f"   Data: {config.max_tokens:,} tokens, seq_len {config.max_seq_len}")

    # Load data
    texts, tokenizer, tokens = load_and_cache_data(config)
    dataset = TextTokenDataset(tokens, config.max_seq_len)

    # Train/val split
    val_size = len(dataset) // 10
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42)
    )

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

    print(f"📊 Dataset: {len(train_dataset)} train, {len(val_dataset)} val samples")

    # Train model
    start_time = time.time()
    model, final_metrics = train_model(config, train_loader, val_loader)
    total_time = time.time() - start_time

    print(f"\n🎉 TRAINING COMPLETED!")
    print(f"⏱️ Total time: {total_time/60:.1f} minutes")
    print(f"🏆 Final Results:")
    print(f"   Validation Loss: {final_metrics['val_loss']:.4f}")
    print(f"   Validation Accuracy: {final_metrics['val_accuracy']:.4f}")
    print(f"   Validation Perplexity: {final_metrics['val_perplexity']:.2f}")

🔍 Device: CUDA
GPU: Tesla T4
Memory: 15.8 GB
🌱 Set all seeds to 42

📋 Model Configuration:
   Architecture: 384d, 6L, 8H, 1536ff
   Training: 2000 steps, batch size 24
   Data: 500,000 tokens, seq_len 512
📦 Loading cached data from data_cache/tokenized_data_2000_500000.pkl
✅ Loaded 2000 documents, 500,000 tokens from cache
📊 Dataset: 449540 train, 49948 val samples

🚀 Training Small model with Muon optimizer
🌱 Set all seeds to 42
  📊 Total parameters: 32,150,976
  Muon parameters: 13,271,040
  AdamW parameters: 18,879,936



Training:   0%|          | 0/2000 [00:00<?, ?it/s, loss=10.8028, acc=0.015, ppl=49156.2, lr=0.00e+00]W0807 09:01:46.185000 1133 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode

Training:  25%|██▌       | 500/2000 [03:00<08:59,  2.78it/s, loss=5.2135, acc=0.221, ppl=183.7, lr=1.00e-02]


Step 500: Val Loss: 4.9942, Val Acc: 0.2329, Val PPL: 147.55
💾 Saved best model with val_loss: 4.9942



Training:  50%|█████     | 1000/2000 [06:08<05:55,  2.82it/s, loss=3.5973, acc=0.356, ppl=36.5, lr=9.86e-03]


Step 1000: Val Loss: 3.1844, Val Acc: 0.4082, Val PPL: 24.15
💾 Saved best model with val_loss: 3.1844



Training:  75%|███████▌  | 1500/2000 [09:16<02:56,  2.83it/s, loss=2.5133, acc=0.495, ppl=12.3, lr=9.54e-03]


Step 1500: Val Loss: 1.8981, Val Acc: 0.6077, Val PPL: 6.67
💾 Saved best model with val_loss: 1.8981



Training: 100%|██████████| 2000/2000 [12:23<00:00,  2.69it/s, loss=1.8079, acc=0.604, ppl=6.1, lr=9.06e-03]

  ⏱️ Training completed in 743.7 seconds


  📊 Final - Loss: 1.1146, Acc: 0.7512, PPL: 3.05
💾 Saved final model to final_model.pt

🎉 TRAINING COMPLETED!
⏱️ Total time: 12.6 minutes
🏆 Final Results:
   Validation Loss: 1.1146
   Validation Accuracy: 0.7512
   Validation Perplexity: 3.05


## 17. Model Loading and Inference

After training, we can load our saved model and use it for text generation.
This section shows how to load the trained model and perform inference.

In [ ]:
import torch
from torch.serialization import add_safe_globals
from transformers import AutoTokenizer

def load_trained_model(model_path: str = "final_model.pt"):
    add_safe_globals([ModelConfig])

    try:
        checkpoint = torch.load(model_path, map_location='cpu')
        config = checkpoint['config']
    except Exception as e:
        print(f"⚠️ Error loading with weights_only=True, trying with weights_only=False...")
        checkpoint = torch.load(model_path, map_location='cpu', weights_only=False)
        config = checkpoint['config']

    model = MinimalLLM(config)
    model.load_state_dict(checkpoint['model_state_dict'])

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()

    print(f"✅ Model loaded successfully")
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"   Device: {device}")

    return model, config

### 🔮 Text Generation Function

This function generates text using a trained language model. Given a prompt, it tokenizes the input and autoregressively samples tokens up to `max_length`. It supports:

- **Temperature scaling** for randomness control, e.g., `0.7` makes output more focused, `1.5` makes it more random.
- **Top-k sampling** to limit candidates to the top `k` most likely tokens, e.g., `top_k=50` narrow down to 50 highest-probability tokens.
- **Top-p (nucleus) sampling** to sample from the smallest set of tokens whose cumulative probability exceeds `p`, i.e. the fewest number of tokens whose combined probabilities add up to at least p (e.g., 90%).

Generation stops early if the EOS token is produced.


In [ ]:
import torch
import torch.nn.functional as F

def generate_text(model: nn.Module, tokenizer, prompt: str, max_length: int = 100,
                 temperature: float = 0.8, top_k: int = 50, top_p: float = 0.9):
    model.eval()
    device = next(model.parameters()).device

    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt').to(device)

    generated_ids = input_ids.clone()

    with torch.no_grad():
        for _ in range(max_length):
            logits = model(generated_ids)
            next_token_logits = logits[0, -1, :] / temperature

            if top_k > 0:
                top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)
                next_token_logits = torch.full_like(next_token_logits, float('-inf'))
                next_token_logits[top_k_indices] = top_k_logits

            if top_p < 1.0:
                sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[1:] = sorted_indices_to_remove[:-1].clone()
                sorted_indices_to_remove[0] = 0
                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                next_token_logits[indices_to_remove] = float('-inf')

            probs = F.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)

            next_token = next_token.unsqueeze(0)
            generated_ids = torch.cat([generated_ids, next_token], dim=1)

            if next_token.item() == tokenizer.eos_token_id:
                break

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

In [ ]:
def interactive_inference(model_path: str = "final_model.pt"):
    """Interactive inference session"""
    print("🤖 Starting interactive inference session")
    print("Type 'quit' to exit")

    # Load model and tokenizer
    model, config = load_trained_model(model_path)

    # Load tokenizer (assuming we have the same one used during training)
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    while True:
        try:
            prompt = input("\n Enter your prompt: ")
            if prompt.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break

            if not prompt.strip():
                continue

            print("🔄 Generating...")
            generated_text = generate_text(
                model, tokenizer, prompt,
                max_length=150,
                temperature=0.8,
                top_k=50,
                top_p=0.9
            )

            print(f"\n Generated text:")
            print(f"📝 {generated_text}")

        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

In [ ]:
def demo_inference(model_path: str = "final_model.pt"):
    """Run a quick demo of the model's capabilities"""
    print("🎭 Running inference demo")

    # Load model and tokenizer
    model, config = load_trained_model(model_path)
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Demo prompts
    demo_prompts = [
        "The future of artificial intelligence",
        "Once upon a time in a distant galaxy",
        "The most important thing to remember is",
        "In the year 2050, technology will",
        "The best way to learn programming is"
    ]

    for i, prompt in enumerate(demo_prompts, 1):
        print(f"\n Demo {i}: '{prompt}'")
        print("-" * 50)

        generated_text = generate_text(
            model, tokenizer, prompt,
            max_length=100,
            temperature=0.7,
            top_k=40,
            top_p=0.85
        )

        print(f"📝 {generated_text}")
        print()

In [ ]:
if __name__ == "__main__":
    # Check if we have a trained model
    import os

    if os.path.exists("final_model.pt"):
        print("🎉 Found trained model! Running demo...")
        demo_inference("final_model.pt")

        # Optionally run interactive session
        response = input("\n🤖 Would you like to try interactive inference? (y/n): ")
        if response.lower() in ['y', 'yes']:
            interactive_inference("final_model.pt")
    else:
        print("⚠️ No trained model found. Please run the training cells first.")
        print("💡 Look for 'final_model.pt' or 'best_model.pt' in your directory.")

🎉 Found trained model! Running demo...
🎭 Running inference demo
 Loading model from final_model.pt
✅ Model loaded successfully
   Parameters: 32,150,976
   Device: cuda

 Demo 1: 'The future of artificial intelligence'
--------------------------------------------------
📝 The future of artificial intelligence, which deals with different variables. For instance, if a case might sound sound sound like the power of data needs to different variables. Similarly, these algorithms often used to generate data needs and problem-quality data.

Another crucial aspect of data is designing data. For example, QoS ensures no single points like a QFD and video packets detailing how much power instead of information. How do they feel like a reward system creates a class, if you could lead to a solution for everyone can


 Demo 2: 'Once upon a time in a distant galaxy'
--------------------------------------------------
📝 Once upon a time in a distant galaxy National Park, I was one day, and I was excited

### 🔮 Text Generation Function

This function generates text using a trained language model. Given a prompt, it tokenizes the input and autoregressively samples tokens up to `max_length`.